*ROUGE (Recall-Oriented Understudy for Gisting Evaluation)* is a set of metrics used to evaluate the quality of automatic summaries of text. It compares a machine-generated summary to a set of human-written reference summaries.

Focus on Recall: ROUGE focuses on recall, which means it measures how much of the important information from the original text is captured in the summary. It doesn't penalize the model for including information not present in the original text (precision).

N-gram Overlap: ROUGE scores are based on the overlap between the n-grams (sequences of n words) in the machine-generated summary and the reference summaries. The most common n-grams used are:

- ROUGE-1 (unigram): This measures the overlap of single words between the summary and references.
- ROUGE-2 (bigram): This measures the overlap of two-word phrases.
- ROUGE-L (Longest Common Subsequence): This considers the longest sequence of words that appears in both the summary and a reference summary.


Score Calculation: ROUGE scores are typically between 0 and 1, with a higher score indicating a better match between the summary and the reference summaries. The score represents the ratio between the number of matching n-grams in the summary and the total number of n-grams in the reference summaries.

In [1]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 1.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.5 MB/s eta 0:00:00ta 0:00:01
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 1.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.7/281.7 kB 10.2 MB/s eta 0:00:00
  DEPRECATION: rouge-score is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for rouge-score ... done

[notice] A new release of pip available: 22.3.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated

In [3]:
from time import sleep
from datetime import datetime

import openai
from openai import OpenAI

import pprint
import difflib
from IPython.display import display, HTML

from rouge_score import rouge_scorer

import os

In [4]:
#os.environ["OPENAI_API_KEY"] = ''

client = OpenAI()

https://cookbook.openai.com/examples/deterministic_outputs_with_the_seed_parameter

The Chat Completions and Completions APIs are non-deterministic by default (which means model outputs may differ from request to request), but now offer some control towards deterministic outputs using a few model level controls.

To receive mostly deterministic outputs across API calls:

- Set the seed parameter to any integer of your choice, but use the same value across requests. For example, 12345.
- Set all other parameters (prompt, temperature, top_p, etc.) to the same values across requests.

In the response, check the system_fingerprint field. The system fingerprint is an identifier for the current combination of model weights, infrastructure, and other configuration options used by OpenAI servers to generate the completion. It changes whenever you change request parameters, or OpenAI updates numerical configuration of the infrastructure serving our models (which may happen a few times a year).

In [5]:
 async def get_chat_response(model_id, user_message, seed = None):
    try:
        response = client.chat.completions.create(
            model = model_id,
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_message}
            ],
            temperature = 0.1,
            seed = seed,
            max_tokens = 256
        )

        response_content = response.choices[0].message.content
        system_fingerprint = response.system_fingerprint
        prompt_tokens = response.usage.prompt_tokens
        completion_tokens = (
            response.usage.total_tokens - response.usage.prompt_tokens
        )

        table = f"""
        <table>
        <tr><th>Response</th><td>{response_content}</td></tr>
        <tr><th>System Fingerprint</th><td>{system_fingerprint}</td></tr>
        <tr><th>Number of prompt tokens</th><td>{prompt_tokens}</td></tr>
        <tr><th>Number of completion tokens</th><td>{completion_tokens}</td></tr>
        </table>
        """
        display(HTML(table))

        return response_content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [6]:
def compare_responses(previous_response, response):

    d = difflib.Differ()

    diff = d.compare(previous_response.split("."), response.split("."))

    diff_exists = False

    diff_table = "<table>"
    for line in diff:
        if line.startswith("- "):
            diff_table += f"<tr style='color: red;'><td>{line}</td></tr>"
            diff_exists = True
        elif line.startswith("+ "):
            diff_table += f"<tr style='color: green;'><td>{line}</td></tr>"
            diff_exists = True
        else:
            diff_table += f"<tr><td>{line}</td></tr>"

    diff_table += "</table>"

    if diff_exists:
        display(HTML(diff_table))
    else:
        print("No differences found.")

### Understanding ROUGE scores

In [8]:
# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer = True)

scorer

In [9]:
# Define the reference text and predicted text
reference_text = "The herd of bisons wandered around near the river"
predicted_text = "The herd of bisons wandered around near the river"

# Compute ROUGE scores
rouge_scores = scorer.score(reference_text, predicted_text)

# Print ROUGE scores
for metric, scores in rouge_scores.items():
    print(f'{metric}:')
    print(f'  Precision: {scores.precision}')
    print(f'  Recall: {scores.recall}')
    print(f'  F1 Score: {scores.fmeasure}')

rouge1:
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
rouge2:
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0
rougeL:
  Precision: 1.0
  Recall: 1.0
  F1 Score: 1.0


In [11]:
# Define the reference text and predicted text
reference_text = "The herd of bisons wandered around near the river"
predicted_text = "The herd of bisons wandered around near the river, trampling on the grass"

# Compute ROUGE scores
rouge_scores = scorer.score(reference_text, predicted_text)

# Print ROUGE scores
for metric, scores in rouge_scores.items():
    print(f'{metric}:')
    print(f'  Precision: {scores.precision}')
    print(f'  Recall: {scores.recall}')
    print(f'  F1 Score: {scores.fmeasure}')

rouge1:
  Precision: 0.6923076923076923
  Recall: 1.0
  F1 Score: 0.8181818181818181
rouge2:
  Precision: 0.6666666666666666
  Recall: 1.0
  F1 Score: 0.8
rougeL:
  Precision: 0.6923076923076923
  Recall: 1.0
  F1 Score: 0.8181818181818181


In [13]:
# Define the reference text and predicted text
reference_text = "The herd of bisons wandered around near the river"
predicted_text = "The monkeys were swinging from tree to tree"

# Compute ROUGE scores
rouge_scores = scorer.score(reference_text, predicted_text)

# Print ROUGE scores
for metric, scores in rouge_scores.items():
    print(f'{metric}:')
    print(f'  Precision: {scores.precision}')
    print(f'  Recall: {scores.recall}')
    print(f'  F1 Score: {scores.fmeasure}')

rouge1:
  Precision: 0.125
  Recall: 0.1111111111111111
  F1 Score: 0.11764705882352941
rouge2:
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0
rougeL:
  Precision: 0.125
  Recall: 0.1111111111111111
  F1 Score: 0.11764705882352941


https://platform.openai.com/docs/models/models

Please note that when you run these you may need to pick a recent model from the list on this page. You don't get a fingerprint if you just use gpt-3.5-turbo. This is likely because that is supposed to pick up the latest model.

##### seed
If specified, our system will make a best effort to sample deterministically, such that repeated requests with the same seed and parameters should return the same result. Determinism is not guaranteed, and you should refer to the system_fingerprint response parameter to monitor changes in the backend.

##### system_fingerprint
This fingerprint represents the backend configuration that the model runs with. It can be used in conjunction with the seed request parameter to understand when backend changes have been made that might impact determinism.This is the indicator on whether users should expect "almost always the same result".

Go to this page to choose a model:

https://platform.openai.com/docs/models

## No seed provided, the responses will be different

In [14]:
model_id = "gpt-4o-2024-05-13"

user_message = """
    Please write an article on the rise of influencers on social media
"""

previous_response = await get_chat_response(
    model_id=model_id,
    user_message=user_message
)

response = await get_chat_response(
    model_id=model_id,
    user_message=user_message
)

Response,"**The Rise of Influencers on Social Media: A Digital Revolution** In the past decade, the landscape of marketing and personal branding has undergone a seismic shift, largely driven by the rise of social media influencers. These digital trendsetters have redefined how brands connect with consumers, how individuals express themselves, and how communities form and interact online. The phenomenon of influencers is not just a fleeting trend but a fundamental change in the way we communicate and consume content. ### The Genesis of Influencers The concept of influencers is not entirely new. Celebrities and public figures have long held sway over public opinion and consumer behavior. However, the advent of social media platforms like Instagram, YouTube, TikTok, and Twitter has democratized influence, allowing ordinary individuals to amass large followings and wield significant power. The early 2010s saw the emergence of the first wave of influencers, primarily on platforms like YouTube and Instagram. These pioneers, often starting as hobbyists, shared content about their passions—be it beauty, fashion, fitness, travel, or gaming. Their authenticity and relatability resonated with audiences, setting them apart from traditional celebrities. ### The Power of Authenticity One of the key drivers behind the rise of influencers is authenticity. Unlike traditional advertisements,"
System Fingerprint,fp_3e7d703517
Number of prompt tokens,32
Number of completion tokens,256


Response,"**The Rise of Influencers on Social Media: A Digital Revolution** In the past decade, the landscape of marketing and personal branding has undergone a seismic shift, driven largely by the rise of social media influencers. These individuals, who have amassed significant followings on platforms like Instagram, YouTube, TikTok, and Twitter, wield considerable power in shaping consumer behavior and cultural trends. The phenomenon of influencers is not just a fleeting trend but a fundamental change in how brands connect with audiences and how individuals express themselves in the digital age. **The Genesis of Influencers** The concept of influencers is not entirely new. Celebrities and public figures have long been used to endorse products and sway public opinion. However, the advent of social media democratized influence, allowing everyday individuals to build personal brands and reach global audiences. The early days of blogging and platforms like MySpace and Facebook set the stage, but it was the visual-centric nature of Instagram and the video capabilities of YouTube that truly catalyzed the influencer movement. **Defining an Influencer** An influencer is typically defined as someone who has the power to affect the purchasing decisions of others because of their authority, knowledge, position, or relationship with their audience. They often specialize in niches such as fashion, beauty, fitness,"
System Fingerprint,fp_3e7d703517
Number of prompt tokens,32
Number of completion tokens,256


In [15]:
compare_responses(previous_response, response)

"- **The Rise of Influencers on Social Media: A Digital Revolution** In the past decade, the landscape of marketing and personal branding has undergone a seismic shift, largely driven by the rise of social media influencers"
? -------
"+ **The Rise of Influencers on Social Media: A Digital Revolution** In the past decade, the landscape of marketing and personal branding has undergone a seismic shift, driven largely by the rise of social media influencers"
? +++++++
"- These digital trendsetters have redefined how brands connect with consumers, how individuals express themselves, and how communities form and interact online"
- The phenomenon of influencers is not just a fleeting trend but a fundamental change in the way we communicate and consume content
"+ These individuals, who have amassed significant followings on platforms like Instagram, YouTube, TikTok, and Twitter, wield considerable power in shaping consumer behavior and cultural trends"
+ The phenomenon of influencers is not just a fleeting trend but a fundamental change in how brands connect with audiences and how individuals express themselves in the digital age
- ### The Genesis of Influencers The concept of influencers is not entirely new
? ^^^^
+ **The Genesis of Influencers** The concept of influencers is not entirely new


In [16]:
# Compute ROUGE scores
rouge_scores = scorer.score(previous_response, response)

# Print ROUGE scores
for metric, scores in rouge_scores.items():
    print(f'{metric}:')
    print(f'  Precision: {scores.precision}')
    print(f'  Recall: {scores.recall}')
    print(f'  F1 Score: {scores.fmeasure}')

rouge1:
  Precision: 0.7028301886792453
  Recall: 0.7412935323383084
  F1 Score: 0.721549636803874
rouge2:
  Precision: 0.44075829383886256
  Recall: 0.465
  F1 Score: 0.4525547445255474
rougeL:
  Precision: 0.4716981132075472
  Recall: 0.4975124378109453
  F1 Score: 0.48426150121065376


## Same model, same seed

Note that the system_fingerprint returned is the same

In [17]:
user_message = """
    Please write an article on the rise of influencers on social media
"""

previous_response = await get_chat_response(
    model_id=model_id,
    user_message=user_message,
    seed=10001
)

response = await get_chat_response(
    model_id=model_id,
    user_message=user_message,
    seed=10001
)

Response,"**The Rise of Influencers on Social Media: A Digital Revolution** In the past decade, the landscape of marketing and personal branding has undergone a seismic shift, driven by the rise of social media influencers. These digital personalities have transformed the way brands connect with consumers, creating a new paradigm in advertising and content creation. From fashion and beauty to technology and travel, influencers have become pivotal players in virtually every industry. This article delves into the factors behind the rise of influencers, their impact on social media, and the future of this dynamic phenomenon. ### The Genesis of Influencers The concept of influencers is not entirely new. Celebrities and public figures have long been used to endorse products and services. However, the advent of social media platforms like Instagram, YouTube, TikTok, and Twitter has democratized influence, allowing ordinary individuals to amass large followings and wield significant sway over their audiences. The rise of influencers can be traced back to the early 2010s, with the proliferation of platforms that enabled users to share content easily and engage with followers. Instagram, launched in 2010, quickly became a hub for visual storytelling, while YouTube, founded in 2005, provided a platform for video content creators to reach global audiences. These platforms, along"
System Fingerprint,fp_3e7d703517
Number of prompt tokens,32
Number of completion tokens,256


Response,"**The Rise of Influencers on Social Media: A Digital Revolution** In the past decade, the landscape of marketing and personal branding has undergone a seismic shift, driven by the rise of social media influencers. These digital personalities have transformed the way brands connect with consumers, creating a new paradigm in advertising and content creation. From fashion and beauty to technology and fitness, influencers have become pivotal players in virtually every industry. This article delves into the factors behind the rise of influencers, their impact on marketing, and the future of this dynamic phenomenon. ### The Genesis of Influencers The concept of influencers is not entirely new. Celebrities and public figures have long been used to endorse products and services. However, the advent of social media platforms like Instagram, YouTube, TikTok, and Twitter has democratized influence, allowing ordinary individuals to amass large followings and wield significant sway over their audiences. The rise of influencers can be traced back to the early 2010s, with the proliferation of platforms that enabled users to share content easily and engage with followers. Instagram, launched in 2010, quickly became a hub for visual storytelling, while YouTube, founded in 2005, provided a platform for video content creators to reach global audiences. These platforms, along with"
System Fingerprint,fp_3e7d703517
Number of prompt tokens,32
Number of completion tokens,256


### The responses below should be the same or almost the same

In [18]:
compare_responses(previous_response, response)

"**The Rise of Influencers on Social Media: A Digital Revolution** In the past decade, the landscape of marketing and personal branding has undergone a seismic shift, driven by the rise of social media influencers"
"These digital personalities have transformed the way brands connect with consumers, creating a new paradigm in advertising and content creation"
"- From fashion and beauty to technology and travel, influencers have become pivotal players in virtually every industry"
? ^^^ ^
"+ From fashion and beauty to technology and fitness, influencers have become pivotal players in virtually every industry"
? ++ ^ ^^
"- This article delves into the factors behind the rise of influencers, their impact on social media, and the future of this dynamic phenomenon"
? ^^^ ^^^^^^^^
"+ This article delves into the factors behind the rise of influencers, their impact on marketing, and the future of this dynamic phenomenon"
? ^^^^^^ ^^
### The Genesis of Influencers The concept of influencers is not entirely new


In [19]:
# Compute ROUGE scores
rouge_scores = scorer.score(previous_response, response)

# Print ROUGE scores
for metric, scores in rouge_scores.items():
    print(f'{metric}:')
    print(f'  Precision: {scores.precision}')
    print(f'  Recall: {scores.recall}')
    print(f'  F1 Score: {scores.fmeasure}')

rouge1:
  Precision: 0.9855072463768116
  Recall: 0.9855072463768116
  F1 Score: 0.9855072463768116
rouge2:
  Precision: 0.9757281553398058
  Recall: 0.9757281553398058
  F1 Score: 0.9757281553398058
rougeL:
  Precision: 0.9855072463768116
  Recall: 0.9855072463768116
  F1 Score: 0.9855072463768116


In [20]:
user_message = """
    Please write an article on the Hermes Birkin bag
"""

previous_response = await get_chat_response(
    model_id=model_id,
    user_message=user_message,
    seed=45678
)

response = await get_chat_response(
    model_id=model_id,
    user_message=user_message,
    seed=45678
)

Response,"**The Iconic Hermes Birkin Bag: A Symbol of Luxury and Craftsmanship** In the world of high fashion, few items command as much reverence and admiration as the Hermes Birkin bag. Synonymous with luxury, exclusivity, and impeccable craftsmanship, the Birkin bag has transcended its status as a mere accessory to become a cultural icon. This article delves into the history, craftsmanship, and enduring allure of the Hermes Birkin bag. ### A Storied Beginning The Birkin bag's origin story is as captivating as the bag itself. In 1983, British actress and singer Jane Birkin found herself seated next to Hermes CEO Jean-Louis Dumas on a flight from Paris to London. As the story goes, Birkin's straw bag spilled its contents, prompting a conversation about the ideal handbag. Inspired by Birkin's needs and suggestions, Dumas sketched a design on an airplane sick bag, laying the foundation for what would become one of the most coveted handbags in the world. ### Craftsmanship: The Heart of the Birkin What sets the Hermes Birkin bag apart from other luxury handbags is the unparalleled craftsmanship that goes into its creation. Each Birkin bag is handmade by a single artisan in Hermes' ateliers in France, a process that"
System Fingerprint,fp_3e7d703517
Number of prompt tokens,30
Number of completion tokens,256


Response,"**The Iconic Hermes Birkin Bag: A Symbol of Luxury and Craftsmanship** In the world of high fashion, few items command as much reverence and admiration as the Hermes Birkin bag. Synonymous with luxury, exclusivity, and impeccable craftsmanship, the Birkin bag has transcended its status as a mere accessory to become a cultural icon. This article delves into the history, craftsmanship, and enduring allure of the Hermes Birkin bag. ### A Storied Beginning The Birkin bag's origin story is as captivating as the bag itself. In 1983, British actress and singer Jane Birkin found herself seated next to Jean-Louis Dumas, the then-chief executive of Hermes, on a flight from Paris to London. As the story goes, Birkin's straw bag tipped over, spilling its contents. This led to a conversation about the ideal handbag, one that combined style with practicality. Inspired by this encounter, Dumas designed a bag that would meet Birkin's needs, and thus, the Birkin bag was born. ### Craftsmanship: A Labor of Love What sets the Hermes Birkin bag apart from other luxury handbags is the unparalleled craftsmanship that goes into its creation. Each Birkin bag is handmade by a single artisan in Hermes' ateliers"
System Fingerprint,fp_3e7d703517
Number of prompt tokens,30
Number of completion tokens,256


### The responses should be the same or almost the same

In [21]:
compare_responses(previous_response, response)

"**The Iconic Hermes Birkin Bag: A Symbol of Luxury and Craftsmanship** In the world of high fashion, few items command as much reverence and admiration as the Hermes Birkin bag"
"Synonymous with luxury, exclusivity, and impeccable craftsmanship, the Birkin bag has transcended its status as a mere accessory to become a cultural icon"
"This article delves into the history, craftsmanship, and enduring allure of the Hermes Birkin bag"
### A Storied Beginning The Birkin bag's origin story is as captivating as the bag itself
"- In 1983, British actress and singer Jane Birkin found herself seated next to Hermes CEO Jean-Louis Dumas on a flight from Paris to London"
? -----------
"+ In 1983, British actress and singer Jane Birkin found herself seated next to Jean-Louis Dumas, the then-chief executive of Hermes, on a flight from Paris to London"
? +++++++++++++++++++++++++++++++++++++
"- As the story goes, Birkin's straw bag spilled its contents, prompting a conversation about the ideal handbag"
"- Inspired by Birkin's needs and suggestions, Dumas sketched a design on an airplane sick bag, laying the foundation for what would become one of the most coveted handbags in the world"
"+ As the story goes, Birkin's straw bag tipped over, spilling its contents"


In [22]:
# Compute ROUGE scores
rouge_scores = scorer.score(previous_response, response)

# Print ROUGE scores
for metric, scores in rouge_scores.items():
    print(f'{metric}:')
    print(f'  Precision: {scores.precision}')
    print(f'  Recall: {scores.recall}')
    print(f'  F1 Score: {scores.fmeasure}')

rouge1:
  Precision: 0.8894472361809045
  Recall: 0.8762376237623762
  F1 Score: 0.8827930174563591
rouge2:
  Precision: 0.7828282828282829
  Recall: 0.7711442786069652
  F1 Score: 0.7769423558897244
rougeL:
  Precision: 0.8291457286432161
  Recall: 0.8168316831683168
  F1 Score: 0.8229426433915212


### Same model different seeds should produce different responses

In [26]:
user_message = """
    Please write an article on the Hermes Birkin bag
"""

previous_response = await get_chat_response(
    model_id=model_id,
    user_message=user_message,
    seed=45678
)

response = await get_chat_response(
    model_id=model_id,
    user_message=user_message,
    seed=890
)

Response,"# The Iconic Hermes Birkin Bag: A Symbol of Luxury and Craftsmanship In the world of high fashion, few items command as much reverence and admiration as the Hermes Birkin bag. Synonymous with luxury, exclusivity, and impeccable craftsmanship, the Birkin bag has transcended its status as a mere accessory to become a cultural icon. This article delves into the history, craftsmanship, and enduring allure of the Hermes Birkin bag. ## A Storied History The Hermes Birkin bag was born out of a chance encounter between British actress and singer Jane Birkin and Hermes CEO Jean-Louis Dumas in 1983. As the story goes, Birkin found herself seated next to Dumas on a flight from Paris to London. When the contents of her overstuffed handbag spilled out, she lamented the lack of a stylish yet practical bag. Inspired by her plight, Dumas sketched a design on an airplane sick bag, and thus, the Birkin bag was conceived. Introduced in 1984, the Birkin bag quickly became a symbol of status and sophistication. Named after Jane Birkin, the bag was designed to be both functional and elegant, featuring a spacious interior, sturdy handles, and a signature lock and key. ## Craft"
System Fingerprint,fp_9cb5d38cf7
Number of prompt tokens,30
Number of completion tokens,256


Response,"# The Iconic Hermes Birkin Bag: A Symbol of Luxury and Craftsmanship In the world of high fashion, few items command as much reverence and admiration as the Hermes Birkin bag. This iconic accessory, synonymous with luxury, exclusivity, and impeccable craftsmanship, has become a coveted status symbol for fashion enthusiasts and celebrities alike. But what is it about the Birkin bag that has propelled it to such legendary status? Let's delve into the history, craftsmanship, and cultural significance of this extraordinary handbag. ## A Storied Beginning The Hermes Birkin bag was born out of a serendipitous encounter between British actress and singer Jane Birkin and Hermes CEO Jean-Louis Dumas in 1983. As the story goes, Birkin found herself seated next to Dumas on a flight from Paris to London. During the flight, she lamented the lack of a stylish yet practical handbag that could accommodate her needs as a young mother. Inspired by her plight, Dumas sketched a design on an airplane sick bag, and thus, the Birkin bag was conceived. ## Craftsmanship: A Labor of Love What sets the Birkin bag apart from other luxury handbags is the unparalleled craftsmanship that goes into its creation. Each Birkin bag is meticulously handcrafted by a"
System Fingerprint,fp_9cb5d38cf7
Number of prompt tokens,30
Number of completion tokens,256


In [24]:
compare_responses(previous_response, response)

"- ### The Timeless Allure of the Hermès Birkin Bag In the world of luxury fashion, few items command as much reverence and admiration as the Hermès Birkin bag"
? ^^^^
"+ **The Timeless Allure of the Hermès Birkin Bag** In the world of luxury fashion, few items command as much reverence and admiration as the Hermès Birkin bag"
? ^^ ++
"Synonymous with opulence, exclusivity, and impeccable craftsmanship, the Birkin bag has transcended its status as a mere accessory to become a cultural icon"
"This article delves into the history, craftsmanship, and enduring appeal of the Hermès Birkin bag"
- #### A Storied Beginning The Birkin bag's inception is the stuff of legend
? -
+ ### A Storied Beginning The Birkin bag's inception is the stuff of legend
"In 1983, British actress and singer Jane Birkin found herself seated next to Jean-Louis Dumas, the then-chief executive of Hermès, on a flight from Paris to London"
"- As the story goes, Birkin's overstuffed straw bag spilled its contents, prompting a conversation about the ideal handbag"


In [25]:
# Compute ROUGE scores
rouge_scores = scorer.score(previous_response, response)

# Print ROUGE scores
for metric, scores in rouge_scores.items():
    print(f'{metric}:')
    print(f'  Precision: {scores.precision}')
    print(f'  Recall: {scores.recall}')
    print(f'  F1 Score: {scores.fmeasure}')

rouge1:
  Precision: 0.9191919191919192
  Recall: 0.91
  F1 Score: 0.9145728643216081
rouge2:
  Precision: 0.8984771573604061
  Recall: 0.8894472361809045
  F1 Score: 0.893939393939394
rougeL:
  Precision: 0.9090909090909091
  Recall: 0.9
  F1 Score: 0.9045226130653266
